## ⚠️ IMPORTANTE: Orden de Ejecución

Ejecuta las celdas en este orden:

1. ▶️ **Celda de Configuración** → Carga el archivo `.env`
2. ▶️ **Celda de Funciones** → Define las funciones de consulta
3. ▶️ **Celda de Consulta** → Ejecuta la consulta con tu SALESID

---

# Conexión OData a Dynamics 365

Este notebook consulta la entidad `PdSalesVSCostProcesseds` de Dynamics 365 filtrando por `SALESID`.

## Requisitos:
1. Crear archivo `.env` con las credenciales (ver `.env.example`)
2. Ejecutar la celda de configuración
3. Ingresar el SALESID a consultar

In [14]:
import os
import json
import time
import requests
from datetime import datetime

# Cache para el token de autenticación
token_cache = {"token": None, "expiry": 0}

def load_env(path=".env"):
    """Carga variables de entorno desde archivo .env"""
    if not os.path.exists(path):
        print(f"⚠️  Archivo {path} no encontrado. Crea uno usando .env.example como referencia.")
        return False
    
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if "=" in line:
                k, v = line.split("=", 1)
                os.environ[k.strip()] = v.strip()
    return True

# Cargar configuración
if load_env():
    print("✅ Configuración cargada desde .env")
else:
    print("❌ No se pudo cargar la configuración")

# Variables de configuración
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
SCOPE_URL = os.getenv("SCOPE_URL")
TOKEN_URL = os.getenv("TOKEN_URL")
BASE_URL = os.getenv("BASE_URL")
ENTITY_NAME = os.getenv("ENTITY_NAME", "PdSalesVSCostProcesseds")
FILTER_FIELD = os.getenv("FILTER_FIELD", "SalesId")

# Validar configuración
config_ok = all([CLIENT_ID, CLIENT_SECRET, SCOPE_URL, TOKEN_URL, BASE_URL])
if config_ok:
    print(f"✅ Entidad configurada: {ENTITY_NAME}")
    print(f"✅ Campo de filtro: {FILTER_FIELD}")
else:
    print("❌ Faltan variables de configuración en el archivo .env")
    missing = []
    if not CLIENT_ID: missing.append("CLIENT_ID")
    if not CLIENT_SECRET: missing.append("CLIENT_SECRET")
    if not SCOPE_URL: missing.append("SCOPE_URL")
    if not TOKEN_URL: missing.append("TOKEN_URL")
    if not BASE_URL: missing.append("BASE_URL")
    print(f"   Variables faltantes: {', '.join(missing)}")

✅ Configuración cargada desde .env
✅ Entidad configurada: PdSalesVSCostProcesseds
✅ Campo de filtro: SalesId


In [15]:
# ============================================================
# SOBRESCRIBIR CONFIGURACIÓN (Opcional)
# ============================================================
# Si modificaste el .env pero no quieres reiniciar el kernel,
# descomenta y ejecuta las líneas siguientes:

# FILTER_FIELD = "SalesId"  # 👈 Nombre EXACTO del campo (case-sensitive)
# print(f"✅ Campo de filtro actualizado a: {FILTER_FIELD}")

# ============================================================

In [16]:
def obtener_token():
    """
    Obtiene un token de autenticación de Azure AD.
    Usa cache para evitar solicitudes innecesarias.
    """
    current_time = time.time()
    
    # Retornar token cacheado si aún es válido
    if token_cache["token"] and token_cache["expiry"] > current_time:
        print("🔐 Usando token cacheado")
        return token_cache["token"]
    
    print("🔐 Solicitando nuevo token...")
    
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": SCOPE_URL
    }
    
    try:
        response = requests.post(TOKEN_URL, headers=headers, data=data, timeout=10)
        
        if response.status_code == 200:
            token_duration = 3600  # 1 hora
            token_cache["token"] = response.json()["access_token"]
            token_cache["expiry"] = current_time + token_duration
            print("✅ Token obtenido exitosamente")
            return token_cache["token"]
        else:
            print(f"❌ Error al obtener token: {response.status_code}")
            print(f"   Respuesta: {response.text[:200]}")
            return None
            
    except Exception as e:
        print(f"❌ Excepción al obtener token: {str(e)}")
        return None

def consultar_por_salesid(salesid):
    """
    Consulta la entidad PdSalesVSCostProcesseds filtrando por SALESID.
    
    Args:
        salesid: El código de venta a buscar
        
    Returns:
        dict: Datos de la respuesta o información de error
    """
    # Verificar que las variables de configuración estén disponibles
    try:
        if not config_ok:
            return {"error": "Configuración incompleta. Revisa el archivo .env"}
    except NameError:
        return {"error": "⚠️ EJECUTA PRIMERO LA CELDA 2 (Configuración) antes de usar esta función"}
    
    # Obtener token
    token = obtener_token()
    if not token:
        return {"error": "No se pudo obtener token de autenticación"}
    
    # Escapar valor para OData (reemplazar ' por '')
    valor = str(salesid).replace("'", "''")
    
    # Construir URL
    base = BASE_URL.rstrip("/")
    url_completa = f"{base}/{ENTITY_NAME}?$filter={FILTER_FIELD} eq '{valor}'"
    
    print(f"\n🔍 Consultando SALESID: {salesid}")
    print(f"   URL: {url_completa}")
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Prefer": "odata.maxpagesize=5000"
    }
    
    try:
        response = requests.get(url_completa, headers=headers, timeout=30)
        
        if response.status_code == 200:
            datos = response.json()
            
            if 'value' in datos:
                num_registros = len(datos['value'])
                print(f"✅ Consulta exitosa: {num_registros} registro(s) encontrado(s)")
                return datos
            else:
                return datos
                
        elif response.status_code == 400:
            # Error de sintaxis OData - probablemente nombre de campo incorrecto
            try:
                error_data = response.json()
                error_msg = error_data.get('error', {}).get('innererror', {}).get('message', '')
                
                if 'Could not find a property named' in error_msg:
                    campo_incorrecto = error_msg.split("'")[1] if "'" in error_msg else FILTER_FIELD
                    print(f"❌ Error 400: Campo '{campo_incorrecto}' no existe en la entidad")
                    print(f"   💡 Sugerencia: Verifica que FILTER_FIELD en .env sea el nombre exacto del campo")
                    print(f"   💡 OData es case-sensitive: 'SalesId' ≠ 'SALESID' ≠ 'salesId'")
                    return {
                        "error": f"Campo '{campo_incorrecto}' no encontrado",
                        "sugerencia": "Verifica FILTER_FIELD en tu archivo .env (case-sensitive)",
                        "status": 400,
                        "detalles": error_msg
                    }
            except:
                pass
            
            print(f"❌ Error en la consulta: {response.status_code}")
            return {
                "error": f"HTTP {response.status_code}",
                "status": response.status_code,
                "mensaje": response.text[:500]
            }
        else:
            print(f"❌ Error en la consulta: {response.status_code}")
            return {
                "error": f"HTTP {response.status_code}",
                "status": response.status_code,
                "mensaje": response.text[:500]
            }
            
    except Exception as e:
        print(f"❌ Excepción durante la consulta: {str(e)}")
        return {"error": "Excepción", "mensaje": str(e)}

def mostrar_resultados(resultado):
    """Muestra los resultados de forma legible"""
    if "error" in resultado:
        print("\n❌ ERROR:")
        print(json.dumps(resultado, ensure_ascii=False, indent=2))
        return
    
    if 'value' in resultado:
        registros = resultado['value']
        
        if not registros:
            print("\n⚠️  No se encontraron registros con ese SALESID")
            return
        
        print(f"\n📊 RESULTADOS ({len(registros)} registro(s)):")
        print("=" * 80)
        
        for i, reg in enumerate(registros, 1):
            print(f"\n--- Registro {i} ---")
            for campo, valor in reg.items():
                if not campo.startswith('@'):  # Omitir metadatos OData
                    print(f"  {campo}: {valor}")
    else:
        print("\n📄 RESPUESTA COMPLETA:")
        print(json.dumps(resultado, ensure_ascii=False, indent=2))

print("\n" + "="*80)
print("✅ Funciones de consulta cargadas correctamente")
print("="*80)


✅ Funciones de consulta cargadas correctamente


In [17]:
# ============================================================
# CONSULTAR POR SALESID
# ============================================================
# Ingresa el SALESID que deseas consultar en la variable:

salesid = "PAT-001260898"  # 👈 Modifica este valor

# ============================================================

if salesid and salesid != "TU-SALESID-AQUI":
    resultado = consultar_por_salesid(salesid)
    mostrar_resultados(resultado)
else:
    print("⚠️  Por favor, ingresa un SALESID válido en la variable 'salesid' arriba ☝️")

🔐 Solicitando nuevo token...
✅ Token obtenido exitosamente

🔍 Consultando SALESID: PAT-001260898
   URL: https://patagonia-prod.operations.dynamics.com/data/PdSalesVSCostProcesseds?$filter=SalesId eq 'PAT-001260898'
✅ Token obtenido exitosamente

🔍 Consultando SALESID: PAT-001260898
   URL: https://patagonia-prod.operations.dynamics.com/data/PdSalesVSCostProcesseds?$filter=SalesId eq 'PAT-001260898'
✅ Consulta exitosa: 1 registro(s) encontrado(s)

📊 RESULTADOS (1 registro(s)):

--- Registro 1 ---
  dataAreaId: pat
  RefCustInvoiceTransRecId: 5640174867
  Qty: -1
  Dev_SalesId: PAT-001258227
  CostAmountAdjustment: 0
  CanalCode: 3010
  Dev_InvoiceId: 39-1487536
  ShippingWarehouseID: LADEHESA
  DiscountCode: 
  LineAmountWithTaxes: -62300
  CostAmountPosted: 22367
  PriceGroupList: PRO30
  SalesPoolId: 
  ItemName: M S STRAIGHT FIT JEANS - REG
  InventSizeId: 30
  TaxAmountMST: 0
  CostAmountPhysical: 22367
  inventSerialId: 
  InventStatusId: DISPONIBLE
  DefaultDimensionDisplayValue:

In [ ]:
# ============================================================
# CONSULTAR MÚLTIPLES SALESID
# ============================================================
# Lista de SALESID a consultar

lista_salesid = [
    "PAT-001260898"
]

# ============================================================

resultados_multiples = {}

for sid in lista_salesid:
    if sid and not sid.startswith("SALESID-"):
        print(f"\n{'='*80}")
        resultado = consultar_por_salesid(sid)
        resultados_multiples[sid] = resultado
        mostrar_resultados(resultado)
        
if not resultados_multiples:
    print("⚠️  Modifica la lista 'lista_salesid' arriba con los SALESID reales a consultar ☝️")

## Consultas Múltiples (Opcional)

Si necesitas consultar varios SALESID, ejecuta la celda siguiente: